In [1]:
# Some imports
import sys
import os, glob
from mfx.processlocalizations import ProcessLocalizations
import re
from matplotlib import pyplot as plt
import math
import numpy as np
from mfx import mfxcolnames as col

In [2]:
# Get file names

def gen_directories( maindir, subdir, keys):
    outdir = {}
    for key in keys:
        outdir[key] = os.path.join(maindir, subdir, key)
    return outdir

def gen_npyfiles(dir_dict):
    npy_files = {}
    for key in dir_dict:
        npy_files[key] = []

    for key in dir_dict:
        for (root, dirs, files) in os.walk(dir_dict[key]):
            if len(dirs) > 0:
                for adir in dirs:
                    npyfile = os.path.join(root, adir, adir +'.npy')
                    if os.path.exists(npyfile):
                        npy_files[key].append(npyfile)
                    
    return npy_files


OUTDIR_LOC = 'C:/Users/apoliti/Desktop/mflux_zarr_tmp_storage/analysis' # Main directory to store zarr files
OUTDIR_REM =  'Z:/siva_minflux/analysis'  # Main directory to store results 
INDIR  = 'Z:/siva_minflux/data'       # main directory of msr file

# Multiple washes with different imager strand
keys = ['Syp_ATG9', 'ZnT3_Syp', 'Syp_Picc']
indir_mwash = gen_directories(INDIR, 'Multiwash', keys)
outdir_mwash = gen_directories(OUTDIR_REM, 'Multiwash', keys)
zarrdir_mwash = gen_directories(OUTDIR_LOC, 'Multiwash', keys) 
npy_mwash = gen_npyfiles(outdir_mwash)

# Wash with a single imager strand
keys = ['Syp', 'ATG9', 'VGLUT1']
indir_swash =  gen_directories(INDIR, 'Single wash', keys)
outdir_swash =  gen_directories(OUTDIR_REM, 'Single wash', keys)
zarrdir_swash =  gen_directories(OUTDIR_LOC, 'Single wash', keys)
npy_swash = gen_npyfiles(outdir_swash)


# Consitency controls. Wash with a single imager strand but multiple times. 
keys = ['VGLUT1_VGLUT1']
indir_cwash =  gen_directories(INDIR, 'Multiwash', keys)
outdir_cwash = gen_directories(OUTDIR_REM, 'Multiwash', keys)
zarrdir_cwash = gen_directories(OUTDIR_LOC, 'Multiwash', keys) 
npy_cwash = gen_npyfiles(outdir_cwash)



In [3]:
def processFile(file_name, eps_range):
    pl = ProcessLocalizations(file_name)
    pl.trim_min_localizations()
    # Split tracks if needed, remove outliers in each track
    pl.cluster_tid(method=pl.CLS_METHOD_BAYES_GMM)

    # find objects that belong to the same cluster according to distance 30 nm
    for eps in eps_range:
            
        pl.DBCLUSTER_EPS_MEAS = eps
        pl.DBCLUSTER_EPS_ALL = eps
        pl.DBCLUSTER_EPS_MERGED_ALL = eps
        pl.DBCLUSTER_EPS_MERGED_MEAS = eps
        pl.log_parameters()
    
        pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all_intersect(col.CLS_ALL)
        summary_dict = pl.summary_per_tid2()
        eps_txt = '%.0f' % (eps*1e9)
        pl.export_csv(summary_dict, file_path=pl.file_path_no_ext() + '_eps' + eps_txt)
        pl.export_vtu(in_dict=summary_dict, coord=col.LTR, 
                      file_path=pl.file_path_no_ext() + '_eps' +  eps_txt)
    

In [ ]:
eps_range = [1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 6e-8, 7e-8, 8e-8]

for label in npy_swash:
    if label == 'VGLUT1':
        for afile in npy_swash[label]:
            processFile(afile, eps_range=eps_range)

In [ ]:
eps_range = [1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 6e-8, 7e-8, 8e-8]
for label in npy_mwash:
    if label == 'Syp_Picc':
        for afile in npy_mwash[label]:
            processFile(afile, eps_range=eps_range)

In [ ]:
eps_range = [1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 6e-8, 7e-8, 8e-8]
for label in npy_cwash:
    for afile in npy_cwash[label]:
        processFile(afile, eps_range=eps_range)

In [6]:
# Export data for presentation
epsv = 4e-8
pl = ProcessLocalizations(npy_mwash['ZnT3_Syp'][1])
pl.export_raw_vtu(pl.loc, coord = col.LTR, file_path=pl.file_path_no_ext() + '_rawdata', unit=col.UNIT_UM)
pl.export_raw_vtu(pl.loc, coord = col.LOC, file_path=pl.file_path_no_ext() + '_rawdata', unit=col.UNIT_UM)
pl.trim_min_localizations()
# Split tracks if needed, remove outliers in each track
pl.cluster_tid(method=pl.CLS_METHOD_BAYES_GMM)
pl.DBCLUSTER_EPS_MEAS = epsv
pl.DBCLUSTER_EPS_ALL = epsv
pl.DBCLUSTER_EPS_MERGED_ALL = epsv
pl.DBCLUSTER_EPS_MERGED_MEAS = epsv
pl.log_parameters()
pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
summary_dict = pl.summary_per_tid2()
eps_txt = '%.0f' % (epsv*1e9)
pl.export_vtu(in_dict=summary_dict, coord=col.LTR, 
                      file_path=pl.file_path_no_ext() + '_eps' +  eps_txt, unit = col.UNIT_UM)
summary_dict_filter = pl.filter_summary_loc(summary_dict, filter_cluster={'name': col.CLS_MERGED_MEAS, 'size': 3})
pl.export_vtu(in_dict=summary_dict_filter, coord=col.LTR, 
                      file_path=pl.file_path_no_ext() + '_eps' +  eps_txt + '_filter_n3', unit = col.UNIT_UM)



export paraview file
export paraview file
2022-09-27 08:47:14,331 [INFO] **** trim_min_localizations ****
220309_ZnT3_P1 Removed 81/361 tracks with less than 3 localizations
220309_Syp_P2 Removed 233/725 tracks with less than 3 localizations



C:\Users\apoliti\Miniconda3\envs\minflux-analysis\lib\site-packages\sklearn\mixture\_base.py:286: ConvergenceWarning: Initialization 3 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


2022-09-27 08:47:22,916 [INFO] **** cluster_tid ****
220309_ZnT3_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 9.49 nm
Processed TID: 46 / 280, Total tracks TID2: 319
220309_Syp_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 9.49 nm
Processed TID: 112 / 492, Total tracks TID2: 585

2022-09-27 08:47:22,920 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis\Multiwash\ZnT3_Syp\220309_ZnT3_Syp_ROI02\220309_ZnT3_Syp_ROI02.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 4e-08
DBCLUSTER_EPS_ALL 4e-08
DBCLUSTER_EPS_MERGED_ALL 4e-08
DBCLUSTER_EPS_MERGED_MEAS 4e-08
ref_error {'se_xy': 2.909006439699532e-09, 'se_z': 2.297593589478866e-09, 'std_xy': 5.038546953104639e-09, 'std_z': 3.979548832121945e-09}
logger <Logger processlocalizations (INFO)>

2022-09-27 08:47:23,463 [INFO] **** cluster_meas ****
220309_ZnT3_P1, Method dbscan, eps: 40.00 nm
Total tracks TID2: 318, total clusters meas: 87
220309_Syp_P2, Method dbscan, eps: 40.0

In [11]:
summary_dict_filter = pl.filter_summary_loc(summary_dict, filter_cluster={'name': col.CLS_MERGED_MEAS, 'size': 3})

In [14]:
npy_mwash['ZnT3_Syp'][1]


'Z:/siva_minflux/analysis\\Multiwash\\ZnT3_Syp\\220309_ZnT3_Syp_ROI02\\220309_ZnT3_Syp_ROI02.npy'